# GeoPanda Practice

# docs: http://geopandas.org/index.html

In [ ]:
import geopandas as gpd
%matplotlib inline

## INSTALL
    * pip install geopandas
    * conda install -c conda-forge geopandas
    
    * pip install geopandas --------------- **FAILED**

In [ ]:
# Test Datasets
gpd.datasets.available

In [ ]:
gpd.datasets.get_path('naturalearth_lowres')

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
world.head()

In [ ]:
world.describe()

In [ ]:
world.plot(figsize=(12, 6))

In [ ]:
#Only one active geometry
world.geometry.name

In [ ]:
#We can also rename this column to “borders”:

world = world.rename(columns={'geometry': 'borders'}).set_geometry('borders')

world.geometry.name

In [ ]:
#Now, we create centroids and make it the geometry:

world['centroid_column'] = world.centroid

world.head()

In [ ]:
world = world.set_geometry('centroid_column')

world.plot(figsize=(12, 6))

### File Format

* Reading and Writing Files Powerfull conversion ... using fiona
* geopandas can read almost any vector-based spatial data format including ESRI shapefile, GeoJSON files and more using the command: gpd.read_file()
* Writing Spatial Data GeoDataFrames can be exported to many different standard formats using the GeoDataFrame.to_file() method. * For a full list of supported formats, type import fiona; fiona.supported_drivers.

In [ ]:
import fiona
fiona.supported_drivers

## Mapping Tools

* geopandas provides a high-level interface to the matplotlib library for making maps. 
* Mapping shapes is as easy as using the plot() method on a GeoSeries or GeoDataFrame.

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

In [ ]:
cities.plot(figsize=(12, 6))

## Chloropleth Maps
* maps where the color of each shape is based on the value of an associated variable
* geopandas makes it easy to create Chloropleth maps
* Simply use the plot command with the column argument set to the column whose values you want used to assign colors.

####  Plot by GDP per capta


In [ ]:
world = world[(world.pop_est>0) & (world.name!="Antarctica")]

world['gdp_per_cap'] = world.gdp_md_est / world.pop_est

world.plot(figsize=(12, 6),column='gdp_per_cap');

In [ ]:
#Choosing colors
#One can also modify the colors used by plot with the cmap option (for a full list of colormaps, see the matplotlib website):

world.plot(figsize=(12, 6),column='gdp_per_cap', cmap='OrRd');

In [ ]:
#if PySAL installed
import pysal
world.plot(figsize=(12, 6),column='gdp_per_cap', cmap='OrRd', scheme='quantiles')

### Maps with Layers
There are two strategies for making a map with multiple layers – one more succinct, and one that is a littel more flexible.

Before combining maps, however, remember to always ensure they share a common CRS (so they will align).



In [ ]:
# Look at capitals
# Note use of standard `pyplot` line style options
cities.plot(marker='*', color='green', markersize=5,figsize=(12, 6));

### CRS : Coordinate Reference Systems

In [ ]:
# Check crs
cities = cities.to_crs(world.crs)

# Now we can overlay over country outlines
# And yes, there are lots of island capitals
# apparently in the middle of the ocean!

In [ ]:
#Method 1

base = world.plot(color='white',figsize=(12, 6))

cities.plot(ax=base, marker='o', color='red', markersize=5,figsize=(12, 6))

In [ ]:
# Method 2: Using matplotlib objects

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 6))

# set aspect to equal. This is done automatically
# when using *geopandas* plot on it's own, but not when
# working with pyplot directly.
ax.set_aspect('equal')

world.plot(ax=ax, color='white')
cities.plot(ax=ax, marker='o', color='red', markersize=5)
plt.show();

## Managing Projections
### CRS - Coordinate Reference Systems

* CRS are important because the geometric shapes in a GeoSeries or GeoDataFrame object are simply a collection of coordinates in an arbitrary space. 
* A CRS tells Python how those coordinates related to places on the Earth.
* CRS are referred to using codes called proj4 strings. You can find the codes for most commonly used projections from www.spatialreference.org or remotesensing.org.


## Re-Projecting
Re-projecting is the process of changing the representation of locations from one coordinate system to another. All projections of locations on the Earth into a two-dimensional plane are distortions, the projection that is best for your application may be different from the projection associated with the data you import. In these cases, data can be re-projected using the to_crs command.


In [ ]:

# load example data
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Check original projection
# (it's Platte Carre! x-y are long and lat)
world.crs


In [ ]:
# Visualize
world.plot(figsize=(12, 6));

In [ ]:
# Reproject to Mercator (after dropping Antartica)
#world[(world.name != "Antarctica") & (world.name != "Fr. S. Antarctic Lands")]

world = world.to_crs({'init': 'epsg:3395'}) # world.to_crs(epsg=3395) would also work

world.plot(figsize=(12, 6))